### In this notebook, I use the Twitter API to pull a sample of tweets including the hashtag '#okboomer'. This code cleans the tweets by reoving emoticons, replacing consecutive non-ASCII characters with a space, and checking for stop words/punctuation. Lastly, it writes several variables of the tweets including the text, author, location, sentiment score (textblob), and hashtags to a csv file. 

In [ ]:
#setup and imports
import os
import pandas as pd
import tweepy
import re
import string
from textblob import TextBlob
import preprocessor as p
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [2]:
#Twitter API credentials
consumer_key = "P1Ci4qLsQB5DyIaQRXcrWpgOd"
consumer_secret ="DUJzsQS3bdJbiKFbc9OqqWzu71KRTlS7kUHSvxKXA4aBUBZ4ux"
access_key = "1169623223878602757-8G0TzARKA2Qo7gmspO4dcK6wYGugBE"
access_secret = "cnzMWVOyRaBHNTSsYrgGLazsvQhqIrbITAWmmLynS0gVf"

In [5]:
#pass twitter credentials to tweepy
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_key, access_secret)
api = tweepy.API(auth)

In [6]:
#set file path
boomer_tweets = "/Users/kpuczkowskyj/Desktop/ADA/applied-data-analytics/My_Work/Text_Data/boomer.csv"

In [7]:
#columns of the csv file
COLS = ['id', 'created_at', 'source', 'original_text','clean_text', 'sentiment','polarity','subjectivity', 'lang',
        'favorite_count', 'retweet_count', 'original_author', 'possibly_sensitive', 'hashtags',
        'user_mentions', 'place', 'place_coord_boundaries']

In [8]:
# declare series of happy and sad emoticons, as well as emoji patterns
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])
 
emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
 
#combine sad and happy emoticons
emoticons = emoticons_happy.union(emoticons_sad)

In [10]:
#function to clean the tweets
def clean_tweets(tweet):
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(tweet)
 
    #clean up after any preprocessing
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
 
 
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
 
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
 
    for w in word_tokens:
        #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
 

In [17]:
#function to pull(tweepy.Cursor) and write the tweets to a csv
def write_tweets(keyword, file):
    if os.path.exists(file):
        df = pd.read_csv(file, header=0)
    else:
        df = pd.DataFrame(columns=COLS)
    for page in tweepy.Cursor(api.search, q=keyword,
                              count=200, include_rts=False, since=2019-1-1).pages(50):
        for status in page:
            new_entry = []
            status = status._json
 
            #only pull English tweets
            if status['lang'] != 'en':
                continue
 
            #replaces the retweet and # likes since last download
            if status['created_at'] in df['created_at'].values:
                i = df.loc[df['created_at'] == status['created_at']].index[0]
                if status['favorite_count'] != df.at[i, 'favorite_count'] or \
                   status['retweet_count'] != df.at[i, 'retweet_count']:
                    df.at[i, 'favorite_count'] = status['favorite_count']
                    df.at[i, 'retweet_count'] = status['retweet_count']
                continue
 
 
           #preprocessing & call clean_tweet to double check
            clean_text = p.clean(status['text'])
 
            filtered_tweet=clean_tweets(clean_text)
 
            #pass textBlob method for sentiment calculations, name polarity & subjectivity variables
            blob = TextBlob(filtered_tweet)
            Sentiment = blob.sentiment

            polarity = Sentiment.polarity
            subjectivity = Sentiment.subjectivity
 
            new_entry += [status['id'], status['created_at'],
                          status['source'], status['text'],filtered_tweet, Sentiment,polarity,subjectivity, status['lang'],
                          status['favorite_count'], status['retweet_count']]
 
            #to append original author of the tweet
            new_entry.append(status['user']['screen_name'])
 
            try:
                is_sensitive = status['possibly_sensitive']
            except KeyError:
                is_sensitive = None
            new_entry.append(is_sensitive)
 
            #save all hashtags and mentions
            hashtags = ", ".join([hashtag_item['text'] for hashtag_item in status['entities']['hashtags']])
            new_entry.append(hashtags)
            mentions = ", ".join([mention['screen_name'] for mention in status['entities']['user_mentions']])
            new_entry.append(mentions)
 
            #get location
            try:
                location = status['user']['location']
            except TypeError:
                location = ''
            new_entry.append(location)
 
            try:
                coordinates = [coord for loc in status['place']['bounding_box']['coordinates'] for coord in loc]
            except TypeError:
                coordinates = None
            new_entry.append(coordinates)
 
            #make panda df, and append to csv
            single_tweet_df = pd.DataFrame([new_entry], columns=COLS)
            df = df.append(single_tweet_df, ignore_index=True)
            csvFile = open(file, 'a' ,encoding='utf-8')
    df.to_csv(csvFile, mode='a', columns=COLS, index=False, encoding="utf-8")
 

In [18]:
#define hashtags we want to search for, and then run it!

boomer_keywords = '#okboomer OR #okayboomer'

write_tweets(boomer_keywords, boomer_tweets)